In [1]:
# Data Processing for HDB Resale Flat Prices
# This notebook combines multiple CSV files and prepares data for visualization

import pandas as pd
import numpy as np
import os
from datetime import datetime

# List of CSV files to process
csv_files = [
    "data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv",
    "data/Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv",
    "data/Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014.csv",
    "data/Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv",
    "data/Resale flat prices based on registration date from Jan-2017 onwards.csv"
]

# Function to read and preprocess each CSV file
def preprocess_csv(file_path):
    print(f"Processing {file_path}...")
    
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Get the relevant columns only (ignore extra columns)
    relevant_columns = [col for col in df.columns if col.lower() in [
        'month', 'town', 'flat_type', 'block', 'street_name', 
        'storey_range', 'floor_area_sqm', 'flat_model', 
        'lease_commence_date', 'resale_price'
    ]]
    
    df = df[relevant_columns]
    
    # Rename columns to ensure consistent lowercase naming
    column_mapping = {col: col.lower() for col in df.columns}
    df = df.rename(columns=column_mapping)
    
    # Convert all string columns to uppercase
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.upper()
    
    return df

# Combine all dataframes
all_data = []
for file in csv_files:
    try:
        df = preprocess_csv(file)
        all_data.append(df)
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Concatenate all dataframes
combined_df = pd.concat(all_data, ignore_index=True)
print(f"Combined data shape: {combined_df.shape}")

# Split month column into year and month
# Format is typically YYYY-MM
combined_df['year'] = combined_df['month'].str.split('-').str[0].astype(int)
combined_df['month_num'] = combined_df['month'].str.split('-').str[1].astype(int)

# Calculate remaining lease years
current_year = datetime.now().year
combined_df['lease_commence_date'] = pd.to_numeric(combined_df['lease_commence_date'], errors='coerce')
combined_df['remaining_lease_year'] = 99 - (combined_df['year'] - combined_df['lease_commence_date'])

# Clean up the data - handle potential issues
# Convert floor_area_sqm to numeric, handling any non-numeric values
combined_df['floor_area_sqm'] = pd.to_numeric(combined_df['floor_area_sqm'], errors='coerce')

# Remove rows with NaN values in important columns
important_cols = ['floor_area_sqm', 'remaining_lease_year', 'resale_price']
combined_df = combined_df.dropna(subset=important_cols)

# Check for any unreasonable values in remaining_lease_year
# Lease should not be negative or unreasonably high
combined_df = combined_df[combined_df['remaining_lease_year'] > 0]
combined_df = combined_df[combined_df['remaining_lease_year'] <= 99]

# Sort by year and month
combined_df = combined_df.sort_values(['year', 'month_num'])

# Display some statistics
print("\nData Statistics:")
print(f"Date Range: {combined_df['year'].min()}-{combined_df['month_num'].min()} to {combined_df['year'].max()}-{combined_df['month_num'].max()}")
print(f"Number of Towns: {combined_df['town'].nunique()}")
print(f"Number of Flat Types: {combined_df['flat_type'].nunique()}")
print(f"Min/Max Floor Area: {combined_df['floor_area_sqm'].min():.1f}/{combined_df['floor_area_sqm'].max():.1f} sqm")
print(f"Min/Max Remaining Lease: {combined_df['remaining_lease_year'].min():.1f}/{combined_df['remaining_lease_year'].max():.1f} years")
print(f"Min/Max Resale Price: ${combined_df['resale_price'].min():,.0f}/${combined_df['resale_price'].max():,.0f}")

# Display unique values for categorical fields
print("\nUnique Flat Types:", combined_df['flat_type'].unique())
print("\nUnique Storey Ranges:", combined_df['storey_range'].unique())
print("\nSample Towns (first 10):", combined_df['town'].unique()[:10])

# Save the processed data to a new CSV file
output_file = "data/combined_hdb_resale_data.csv"
combined_df.to_csv(output_file, index=False)
print(f"\nData saved to {output_file}")

# Display sample of final dataset
print("\nSample of processed data:")
combined_df.head()

Processing data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv...
Processing data/Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv...
Processing data/Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014.csv...
Processing data/Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv...
Processing data/Resale flat prices based on registration date from Jan-2017 onwards.csv...
Combined data shape: (947852, 10)

Data Statistics:
Date Range: 1990-1 to 2025-12
Number of Towns: 27
Number of Flat Types: 8
Min/Max Floor Area: 28.0/366.7 sqm
Min/Max Remaining Lease: 41.0/99.0 years
Min/Max Resale Price: $5,000/$1,600,000

Unique Flat Types: ['1 ROOM' '3 ROOM' '4 ROOM' '5 ROOM' '2 ROOM' 'EXECUTIVE'
 'MULTI GENERATION' 'MULTI-GENERATION']

Unique Storey Ranges: ['10 TO 12' '04 TO 06' '07 TO 09' '01 TO 03' '13 TO 15' '19 TO 21'
 '16 TO 18' '25 TO 27' '22 TO 24' '28 TO 30' '31 TO 33' '40 TO 42'
 '37 TO 39' '34 TO 36' '06 TO 10

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,year,month_num,remaining_lease_year
0,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,9000.0,1990,1,86
1,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,04 TO 06,31.0,IMPROVED,1977,6000.0,1990,1,86
2,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,10 TO 12,31.0,IMPROVED,1977,8000.0,1990,1,86
3,1990-01,ANG MO KIO,1 ROOM,309,ANG MO KIO AVE 1,07 TO 09,31.0,IMPROVED,1977,6000.0,1990,1,86
4,1990-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,04 TO 06,73.0,NEW GENERATION,1976,47200.0,1990,1,85


In [3]:
# in data/combined_hdb_resale_data.csv,flat_type contains a redundancy: MULTI-GENERATION and MULTI GENERATION

# Fix the redundancy in flat_type by replacing MULTI GENERATION with MULTI-GENERATION
combined_df['flat_type'] = combined_df['flat_type'].str.replace('MULTI GENERATION', 'MULTI-GENERATION')

# Filter floor_area_sqm beyond 250 sqm
combined_df = combined_df[combined_df['floor_area_sqm'] <= 250]

# Save the updated data to a new CSV file
output_file = "data/combined_hdb_resale_data_filtered.csv"
combined_df.to_csv(output_file, index=False)
print(f"\nData saved to {output_file}")


Data saved to data/combined_hdb_resale_data_filtered.csv
